

# 决策树
***
## 天气决策树
我们通过判断$X_{i}$(weather、humidity、wind、temperature)的组合来确定最终是否合适出行（suitable）。

|id|weather|humidity|wind|temperature|suitable|
|--|-------|--------|----|-----------|--------|
|1 |sunny  |low    |yes |cold     |yes    |
|2 |sunny  |normal  |yes |high     |no     |
|3 |sunny  |high   |no  |high     |no     |
|4 |sunny  |high   |yes |cold     |no     |
|5 |sunny  |normal  |no  |high     |no     |
|6 |rainy  |low    |yes |cold     |no     |
|7 |rainy  |normal  |yes |cold     |no     |
|8 |rainy  |high   |no  |cold     |yes    |
|9 |rainy  |high   |yes |cold     |no     |
|10|rainy  |high   |no  |high     |yes     |
|11|rainy  |normal  |no  |cold     |no     |
|12|cloudy |low    |yes |high     |yes     |
|13|cloudy |high   |yes |cold     |no    |
|14|cloudy |high   |no  |cold     |no    |
|15|cloudy |normal  |no  |high     |yes    |

***
## 信息熵
上表中合适出行的天数为5天，不合适出行10天。信息熵描述了信息的不确定性，此例中便是suitable的两种状态的不确定性，信息熵越大，不确定性越大，若得到确定信息需要输入的信息越多。

$$E=-\sum_{i=1}^{N}P_{i}log_{2}P_{i}$$

N是状态数，此例中便是2，对应是否合适出行的yes和no。其中$P_{yes}=\frac{1}{3}$,
$P_{no}=\frac{2}{3}$。因此，其信息熵

$$E=-P_{yes}logP_{yes} - P_{no}logP_{no}=-\frac{1}{3}\times log\frac{1}{3} - \frac{2}{3}\times log\frac{2}{3} = 0.918295834054$$

***
## 信息增益
当选择某个属性$X$（上表中对应的一列）时，确定该属性值后能减少的信息熵的量，就是信息增益，即某属性值确定后，能够给最后的决策减少的不确定性的量。以本例来说，就是当确定了某个属性（例如是否有风wind）后，比如确定了是无风（no），更容易做出最后是否合适出行的结论，因为此时系统的不确定性减少了。你得到了一部分信息输入（是否有风），因而减少了下一步的不确定性，因而系统比起初始状态来说信息熵更小了，而小了多少，就是该属性$X$的信息增益。信息增益越大，给最后做出决策带来的信息量越大。举个极端的栗子，假如上表中的无风都适合出行，有风都不适合出行，那么只要上表中wind值确定了，suitable就确定了，此时wind属性就是信息增益最大的属性，因为它完全消除了系统的不确定性。那么，如何度量信息增益呢？某属性的信息增益等于总的信息熵减去该属性在作出决策方面的信息熵。

$$G(A) = E - \sum_{i = 0}^{m}\frac{A_{i}}{S}E(A_{i})$$

E是系统的信息熵，G(A)是属性A的信息增益。m是A的各个分类的个数（比如wind为2，weather为3),S是总的样本量（本例为15），$A_{i}$是属性A的i分类下的样本量（例如wind的no为5，weather的sunny为5）。$E(A_{i}$为该属性分类在作出决策方面的信息熵。回到刚刚那个极端的例子，若wind的5个no全是合适出门，10个yes全是不合适。则wind的信息增益为：

$$G(wind) = 0.918 - \frac{5}{15}E(A_{no}) - \frac{10}{15}E(A_{yes})$$

$$E(A_{no}) = -P_{yes}logP_{yes} - P_{no}logP_{no}= -1 \times log1 - 0 \times log 0 = 0$$

$$E(A_{yes}) =  -P_{yes}logP_{yes} - P_{no}logP_{no}= -0 \times log0 - 1 \times log 1 = 0 $$

$$G(wind) = 0.918 - 0 - 0 = 0.918$$

也就是说其信息增益=信息熵，符合刚才说的完全消除了系统的不确定性。

为了能最快最准确地得到最后的决策，信息增益越大的属性就应该越靠近根节点。如果是刚刚那个极端的情况，wind就是根节点，而且这一个节点就足够判断最后的结论了，其他的节点都可以减去了。

In [68]:
import math

def getEntropy(dataset, resultColumnName):
    resultList = dataset[resultColumnName].values.tolist()
    sampleNum = float(len(resultList))
    labelCount = {}
    for data in resultList:
        if data not in labelCount.keys():
            labelCount[data] = 0
        labelCount[data] += 1
    entropy = 0.0
    for label in labelCount:
        probability = labelCount[label] / sampleNum
        entropy -= probability * math.log(probability, 2)
    return entropy
getEntropy(getDataset("C:/users/n4193/desktop/decisiontree.csv"), 'suitable')

0.9182958340544896

In [72]:
def getDataset(s):
    '''返回DataFrame'''
    df = pd.read_csv(s)
    return df.ix[:, 1:]

'\xe8\xbf\x94\xe5\x9b\x9eDataFrame'

In [106]:
def getColumnNames(dataSet):
    '''以list返回列名'''
    
    return df.columns.tolist()
df=getDataset("C:/users/n4193/desktop/decisiontree.csv")
getColumnNames(df)

['weather', 'humidity', 'wind', 'temperature', 'suitable']

In [85]:
def splitNode(dataSet, colName, value):
    '''返回分叉后的一个分支,DataFrame类型。传入一个DataFrame样本dataSet，
    待分支的节点属性colName，和该分支的属性值value'''
    
    branch =  dataSet[dataSet[colName].isin([value])]
    del branch[colName]
    return branch

splitNode(getDataset("C:/users/n4193/desktop/decisiontree.csv"), 'wind', 'yes')

,weather,humidity,temperature,suitable
0,sunny,low,cold,yes
1,sunny,normal,high,no
3,sunny,high,cold,no
5,rainy,low,cold,no
6,rainy,normal,cold,no
8,rainy,high,cold,no
11,cloudy,low,high,yes
12,cloudy,high,cold,no


In [119]:
def getColumnNames(dataSet):
    '''以list返回列名'''
    
    return df.columns.tolist()


def chooseFeatureToSplit(dataSet):
    '''返回最合适分支的属性的列名'''
    
    colNameList = getColumnNames(dataSet)
    resultColumnName = colNameList[-1]
    colInfoGain = 0.0
    feature = colNameList[0]
    for col in colNameList:
        
        #最后一列是结果，不计算
        if col == resultColumnName:
            continue
            
        valueSet = set(dataSet[col].values)
        columnEntropy = 0.0;
        
        for branch in valueSet:
            df = dataSet[dataSet[col].isin([branch])]
            probCoefficient = len(df) / float(len(dataSet))
            columnEntropy += probCoefficient * getEntropy(df, resultColumnName)
            
        currentInfoGain = getEntropy(dataSet, resultColumnName) - columnEntropy
        if currentInfoGain > colInfoGain:
            colInfoGain = currentInfoGain
            feature = col
            
    return feature
df = splitNode(getDataset("C:/users/n4193/desktop/decisiontree.csv"), 'wind', 'yes')
getColumnNames(df)
chooseFeatureToSplit(df)

'humidity'

import pandas as pd
df = pd.read_csv("C:/users/n4193/desktop/decisiontree.csv")

set(df['wind'].values)

In [109]:
import math
2 - math.log(3, 2) * 3 / 4

0.8112781244591327